In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import os
os.makedirs('/root/.kaggle',exist_ok = True)
!cp /content/gdrive/My\ Drive/DeepLearning/kaggle/kaggle.json /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c the-nature-conservancy-fisheries-monitoring

In [0]:
!kaggle datasets download -d ningyue1/vgg-weight

In [0]:
# 解压预训练权重
! unzip /content/vgg-weight.zip

In [0]:
# 解压训练集数据
! unzip /content/train.zip

In [0]:
# 解压第一部分测试集数据
!unzip /content/test_stg1.zip

In [0]:
# 解压第二部分测试集数据
!7za x test_stg2.7z

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os,cv2
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

In [0]:
# 定义取图片的路径
TRAIN_DIR = '/content/train/'
TEST_DIR = '/content/test_stg2/'
WEIGHT_DIR = '/content/vgg16_weights.npz'
FISH_CLASSES = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

In [0]:
# 获取图片的路径方法
def get_images(data_dir,fish):
    """Load files from train folder"""
    fish_dir = data_dir+'{}'.format(fish)
    images = [fish+'/'+im for im in os.listdir(fish_dir)]
    return images
# 将图片读入内存
def read_image(src):
    """Read and resize individual images"""
    im = cv2.imread(src, cv2.IMREAD_COLOR)
    im = cv2.resize(im, (224, 224), interpolation=cv2.INTER_CUBIC)
    return im

In [0]:
# 获取训练数据
files = []
y_all = []

for fish in FISH_CLASSES:
    fish_files = get_images(TRAIN_DIR,fish)
    files.extend(fish_files)
    
    y_fish = np.tile(fish, len(fish_files))
    y_all.extend(y_fish)
    print("{0} photos of {1}".format(len(fish_files), fish))
    
y_all = np.array(y_all)

In [0]:
# 将所有图片加载到内存中
X = np.ndarray((len(files), 224, 224, 3))

for i, im in enumerate(files): 
    X[i] = read_image(TRAIN_DIR+im)
    if i%1000 == 0: print('Processed {} of {}'.format(i, len(files)))
print(X.shape)

In [0]:
# 转化为one-hot编码
y = LabelEncoder().fit_transform(y_all)
y = np_utils.to_categorical(y)

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                    test_size=0.2, random_state=23, 
                                                    stratify=y)

In [0]:
print(X_train.shape,X_valid.shape,y_train.shape,y_valid.shape)

(3021, 224, 224, 3) (756, 224, 224, 3) (3021, 8) (756, 8)


In [0]:
# 加载预训练的权重
pre_weight = np.load(WEIGHT_DIR,allow_pickle=True)

In [0]:
list(pre_weight.keys())

['conv4_3_W',
 'conv5_1_b',
 'conv1_2_b',
 'conv5_2_b',
 'conv1_1_W',
 'conv5_3_b',
 'conv5_2_W',
 'conv5_3_W',
 'conv1_1_b',
 'fc7_b',
 'conv5_1_W',
 'conv1_2_W',
 'conv3_2_W',
 'conv4_2_b',
 'conv4_1_b',
 'conv3_3_W',
 'conv2_1_b',
 'conv3_1_b',
 'conv2_2_W',
 'fc6_b',
 'fc8_b',
 'conv4_3_b',
 'conv2_2_b',
 'fc6_W',
 'fc8_W',
 'fc7_W',
 'conv3_2_b',
 'conv4_2_W',
 'conv3_3_b',
 'conv3_1_W',
 'conv2_1_W',
 'conv4_1_W']

In [0]:
# 学习参数
learning_rate = 0.003
num_epochs = 75
batch_size = 64
batch_num = 47

# 网络参数
dropout_rate = 0.5
num_classes = 8

In [0]:
# 构建模型
x = tf.placeholder(tf.float32, [None, 224, 224, 3])
y = tf.placeholder(tf.int32, [None, num_classes])

W_conv = {
    'conv1_1': tf.Variable(pre_weight['conv1_1_W'],trainable=False),
    'conv1_2': tf.Variable(pre_weight['conv1_2_W'],trainable=False),
    'conv2_1': tf.Variable(pre_weight['conv2_1_W'],trainable=False),
    'conv2_2': tf.Variable(pre_weight['conv2_2_W'],trainable=False),
    'conv3_1': tf.Variable(pre_weight['conv3_1_W'],trainable=False),
    'conv3_2': tf.Variable(pre_weight['conv3_2_W'],trainable=False),
    'conv3_3': tf.Variable(pre_weight['conv3_3_W'],trainable=False),
    'conv4_1': tf.Variable(pre_weight['conv4_1_W'],trainable=False),
    'conv4_2': tf.Variable(pre_weight['conv4_2_W'],trainable=False),
    'conv4_3': tf.Variable(pre_weight['conv4_3_W'],trainable=False),
    'conv5_1': tf.Variable(pre_weight['conv5_1_W'],trainable=False),
    'conv5_2': tf.Variable(pre_weight['conv5_2_W'],trainable=False),
    'conv5_3': tf.Variable(pre_weight['conv5_3_W'],trainable=False),
    'fc6': tf.Variable(tf.truncated_normal([7*7*512, 4096],stddev=0.001)),
    'fc7': tf.Variable(tf.truncated_normal([4096, 4096], stddev=0.001)),
    'fc8': tf.Variable(tf.truncated_normal([4096, num_classes], stddev=0.001))
}
b_conv = {
    'conv1_1': tf.Variable(pre_weight['conv1_1_b'],trainable=False),
    'conv1_2': tf.Variable(pre_weight['conv1_2_b'],trainable=False),
    'conv2_1': tf.Variable(pre_weight['conv2_1_b'],trainable=False),
    'conv2_2': tf.Variable(pre_weight['conv2_2_b'],trainable=False),
    'conv3_1': tf.Variable(pre_weight['conv3_1_b'],trainable=False),
    'conv3_2': tf.Variable(pre_weight['conv3_2_b'],trainable=False),
    'conv3_3': tf.Variable(pre_weight['conv3_3_b'],trainable=False),
    'conv4_1': tf.Variable(pre_weight['conv4_1_b'],trainable=False),
    'conv4_2': tf.Variable(pre_weight['conv4_2_b'],trainable=False),
    'conv4_3': tf.Variable(pre_weight['conv4_3_b'],trainable=False),
    'conv5_1': tf.Variable(pre_weight['conv5_1_b'],trainable=False),
    'conv5_2': tf.Variable(pre_weight['conv5_2_b'],trainable=False),
    'conv5_3': tf.Variable(pre_weight['conv5_3_b'],trainable=False),
    'fc6': tf.Variable(tf.constant(0.1,dtype=tf.float32, shape=[4096])),
    'fc7': tf.Variable(tf.constant(0.1,dtype=tf.float32, shape=[4096])),
    'fc8': tf.Variable(tf.constant(0.1,dtype=tf.float32, shape=[num_classes]))
}

In [0]:
# 第一层
conv1_1 = tf.nn.conv2d(x,W_conv['conv1_1'],strides=[1, 1, 1, 1], padding = 'SAME')
conv1_1 = tf.nn.bias_add(conv1_1,b_conv['conv1_1'])
conv1_1 = tf.nn.relu(conv1_1)
conv1_2 = tf.nn.conv2d(conv1_1,W_conv['conv1_2'],strides=[1, 1, 1, 1], padding = 'SAME')
conv1_2 = tf.nn.bias_add(conv1_2,b_conv['conv1_2'])
conv1_2 = tf.nn.relu(conv1_2)
pool1 = tf.nn.max_pool(conv1_2, [1, 2, 2, 1], [1, 2, 2, 1], padding="VALID")

# 第二层
conv2_1 = tf.nn.conv2d(pool1,W_conv['conv2_1'],strides=[1, 1, 1, 1], padding = 'SAME')
conv2_1 = tf.nn.bias_add(conv2_1,b_conv['conv2_1'])
conv2_1 = tf.nn.relu(conv2_1)
conv2_2 = tf.nn.conv2d(conv2_1,W_conv['conv2_2'],strides=[1, 1, 1, 1], padding = 'SAME')
conv2_2 = tf.nn.bias_add(conv2_2,b_conv['conv2_2'])
conv2_2 = tf.nn.relu(conv2_2)
pool2 = tf.nn.max_pool(conv2_2, [1, 2, 2, 1], [1, 2, 2, 1], padding="VALID")

# 第三层
conv3_1 = tf.nn.conv2d(pool2,W_conv['conv3_1'],strides=[1, 1, 1, 1], padding = 'SAME')
conv3_1 = tf.nn.bias_add(conv3_1,b_conv['conv3_1'])
conv3_1 = tf.nn.relu(conv3_1)
conv3_2 = tf.nn.conv2d(conv3_1,W_conv['conv3_2'],strides=[1, 1, 1, 1], padding = 'SAME')
conv3_2 = tf.nn.bias_add(conv3_2,b_conv['conv3_2'])
conv3_2 = tf.nn.relu(conv3_2)
conv3_3 = tf.nn.conv2d(conv3_2,W_conv['conv3_3'],strides=[1, 1, 1, 1], padding = 'SAME')
conv3_3 = tf.nn.bias_add(conv3_3,b_conv['conv3_3'])
conv3_3 = tf.nn.relu(conv3_3)
pool3 = tf.nn.max_pool(conv3_3, [1, 2, 2, 1], [1, 2, 2, 1], padding="VALID")

# 第四层
conv4_1 = tf.nn.conv2d(pool3,W_conv['conv4_1'],strides=[1, 1, 1, 1], padding = 'SAME')
conv4_1 = tf.nn.bias_add(conv4_1,b_conv['conv4_1'])
conv4_1 = tf.nn.relu(conv4_1)
conv4_2 = tf.nn.conv2d(conv4_1,W_conv['conv4_2'],strides=[1, 1, 1, 1], padding = 'SAME')
conv4_2 = tf.nn.bias_add(conv4_2,b_conv['conv4_2'])
conv4_2 = tf.nn.relu(conv4_2)
conv4_3 = tf.nn.conv2d(conv4_2,W_conv['conv4_3'],strides=[1, 1, 1, 1], padding = 'SAME')
conv4_3 = tf.nn.bias_add(conv4_3,b_conv['conv4_3'])
conv4_3 = tf.nn.relu(conv4_3)
pool4 = tf.nn.max_pool(conv4_3, [1, 2, 2, 1], [1, 2, 2, 1], padding="VALID")

# 第五层
conv5_1 = tf.nn.conv2d(pool4,W_conv['conv5_1'],strides=[1, 1, 1, 1], padding = 'SAME')
conv5_1 = tf.nn.bias_add(conv5_1,b_conv['conv5_1'])
conv5_1 = tf.nn.relu(conv5_1)
conv5_2 = tf.nn.conv2d(conv5_1,W_conv['conv5_2'],strides=[1, 1, 1, 1], padding = 'SAME')
conv5_2 = tf.nn.bias_add(conv5_2,b_conv['conv5_2'])
conv5_2 = tf.nn.relu(conv5_2)
conv5_3 = tf.nn.conv2d(conv5_2,W_conv['conv5_3'],strides=[1, 1, 1, 1], padding = 'SAME')
conv5_3 = tf.nn.bias_add(conv5_3,b_conv['conv5_3'])
conv5_3 = tf.nn.relu(conv5_3)
pool5 = tf.nn.max_pool(conv5_3, [1, 2, 2, 1], [1, 2, 2, 1], padding="VALID")

reshape = tf.reshape(pool5, [-1, 7*7*512])

fc6 = tf.add(tf.matmul(reshape, W_conv['fc6']), b_conv['fc6'])
fc6 = tf.nn.relu(fc6)
fc6 = tf.nn.dropout(fc6,0.5)

fc7 = tf.add(tf.matmul(fc6, W_conv['fc7']), b_conv['fc7'])
fc7 = tf.nn.relu(fc7)
fc7 = tf.nn.dropout(fc7,0.5)

fc8 = tf.add(tf.matmul(fc7, W_conv['fc8']), b_conv['fc8'])
fc8 = tf.nn.relu(fc8)
fc8 = tf.nn.dropout(fc8,0.5)

In [0]:
# 定义损失
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc8, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
# 评估模型
correct_pred = tf.equal(tf.argmax(fc8, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [0]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_epochs):
        for j in range(batch_num):
            X_train_data = X_train[j*batch_size:(j+1)*batch_size,:,:,:]
            y_train_data = y_train[j*batch_size:(j+1)*batch_size,:]
            sess.run(optimizer,feed_dict={x:X_train_data,y:y_train_data})
            if j % 10 ==0:  
                _loss = sess.run(loss, feed_dict={x:X_train_data,y:y_train_data})
                train_accuracy = sess.run(accuracy, feed_dict={x:X_train_data, y: y_train_data})
                print('Epoch: {:>2} Batch {:>5} Loss: {:>10.6f} Accuracy: {:.6f}'.format(i, j, _loss, train_accuracy))

Epoch:  0 Batch     0 Loss:   2.047429 Accuracy: 0.328125
Epoch:  0 Batch    10 Loss:   1.872801 Accuracy: 0.343750
Epoch:  0 Batch    20 Loss:   1.795046 Accuracy: 0.421875
Epoch:  0 Batch    30 Loss:   1.969324 Accuracy: 0.328125
Epoch:  0 Batch    40 Loss:   1.853640 Accuracy: 0.328125
Epoch:  1 Batch     0 Loss:   1.797387 Accuracy: 0.375000
Epoch:  1 Batch    10 Loss:   1.824742 Accuracy: 0.390625
Epoch:  1 Batch    20 Loss:   1.736486 Accuracy: 0.375000
Epoch:  1 Batch    30 Loss:   1.834299 Accuracy: 0.359375
Epoch:  1 Batch    40 Loss:   1.737167 Accuracy: 0.421875
Epoch:  2 Batch     0 Loss:   1.727727 Accuracy: 0.421875
Epoch:  2 Batch    10 Loss:   1.672629 Accuracy: 0.437500
Epoch:  2 Batch    20 Loss:   1.530337 Accuracy: 0.343750
Epoch:  2 Batch    30 Loss:   1.834884 Accuracy: 0.296875
Epoch:  2 Batch    40 Loss:   1.818706 Accuracy: 0.437500
Epoch:  3 Batch     0 Loss:   1.484197 Accuracy: 0.437500
Epoch:  3 Batch    10 Loss:   1.560740 Accuracy: 0.468750
Epoch:  3 Batc

In [0]:
import gc
del X
gc.collect()

In [0]:
 test_file_path = [TEST_DIR+im for im in os.listdir(TEST_DIR)]

In [0]:
test_preds = np.zeros((12153,8))

In [0]:
test = np.ndarray((100, 224, 224, 3))

In [0]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(121):
      print(i)
      for j in range(100):
        test[j] = read_image(test_file_path[i*100+j])
      t_pred = sess.run(tf.nn.softmax(fc8),feed_dict={x:test})
      test_preds[i*100:(i+1)*100,:] = t_pred
    for i in range(53):
      test[j] = read_image(test_file_path[121*100+i])
    t_pred = sess.run(tf.nn.softmax(fc8),feed_dict={x:test[0:53]})
    test_preds[121*100:121*100+53,:] = t_pred

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120


In [0]:
test_file_path1 = ['test_stg2/'+im for im in os.listdir(TEST_DIR)]

In [0]:
submission = pd.DataFrame(test_preds, columns=FISH_CLASSES)
submission.insert(0, 'image', test_file_path1)
submission.head()


submission.to_csv('submisision2.csv',index=None)


In [0]:
submission.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,test_stg2/image_05356.jpg,0.118372,0.118372,0.143498,0.118372,0.146273,0.118372,0.118372,0.118372
1,test_stg2/image_03402.jpg,0.117797,0.117797,0.148447,0.117797,0.117797,0.117797,0.144773,0.117797
2,test_stg2/image_06302.jpg,0.131480,0.138973,0.136623,0.134238,0.108405,0.133471,0.108405,0.108405
3,test_stg2/image_03224.jpg,0.131143,0.131861,0.131238,0.127301,0.107340,0.107340,0.132859,0.130917
4,test_stg2/image_07578.jpg,0.108042,0.128779,0.108042,0.134476,0.131037,0.131103,0.128885,0.129635


In [0]:
test_files = [im for im in os.listdir('/content/test_stg1')]
test = np.ndarray((len(test_files), 224, 224, 3))
for i, im in enumerate(test_files): 
    test[i] = read_image('/content/test_stg1/'+im)

In [0]:
test_preds = np.zeros((1000,8))
with tf.Session() as sess:
    sess.run(init)
    for i in range(10):
        print(i)
        t_pred = sess.run(tf.nn.softmax(fc8),feed_dict={x:test[i*100:(i+1)*100,:,:,:]})
        test_preds[i*100:(i+1)*100,:] = t_pred



submission = pd.DataFrame(test_preds, columns=FISH_CLASSES)
submission.insert(0, 'image', test_files)
submission.head()


submission.to_csv('submisision1.csv',index=None)


0
1
2
3
4
5
6
7
8
9
